## Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da aula passada, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada.

Na camada de auto-atenção, deve-se implementar (vide slide 34):
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Camada de feed forward (2-layer MLP)

Instrucões:
- É necessário fazer duas implementações da camada de auto-atenção: uma usando laços (ineficiente, mas fácil de entender) e outra matricial (eficiente mas difícil de entender). Usar slide 36 como referência.

- Fazer um assert para garantir que o resultado das duas implementações é exatamente igual.

- No treinamento, usar apenas a implementação matricial.

## Imports

In [1]:
import os
import sys
import random
import torch.nn as nn
import torch.nn.functional as F
import time
from sklearn.model_selection import train_test_split
from torch import nn

## Variáveis Globais e Inicialização

In [2]:
# Global variables

# Vocabulary
vocab_size = 5000
context_size = 5
pattern = r'\w+|[,;.:!?\']'

# Training
batch_size = 128
epochs = 10
lr = 0.1

# Model
embedding_dim = 256
hidden_dim = 128

In [3]:
# Colab environment
IN_COLAB = 'google.colab' in sys.modules

if (IN_COLAB):
    %pip install colorama

    # Google Drive
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    project_folder="/content/drive/MyDrive/Classes/IA024/Aula_2_3"
    os.chdir(project_folder)
    !ls -la

## Faz download e carrega o dataset

In [4]:
# Check if download is necessary
if not os.path.exists("67724.txt.utf-8"):
    print("Downloading Gutenberg texts")

    !wget https://www.gutenberg.org/ebooks/67724.txt.utf-8
    !wget https://www.gutenberg.org/ebooks/67725.txt.utf-8

In [5]:
text = open("67724.txt.utf-8","r").read()
text += open("67725.txt.utf-8","r").read()

paragraphs = text.split("\n\n")

len(paragraphs)

4969

In [6]:
# Checking the text
print(paragraphs[0])

The Project Gutenberg eBook of O Guarany: romance brazileiro, Vol. 1 (of 2)
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.


In [7]:
cleaned_paragraphs = [paragraph.replace("\n", " ") for paragraph in paragraphs if paragraph.strip()]

# Print 5 random paragraphs
num_paragraphs = len(cleaned_paragraphs)
for i in range(0,5):
    idx = random.randrange(num_paragraphs)
    print(f"{cleaned_paragraphs[idx]}\n")

print("Number of paragraphs: " + str(num_paragraphs))

len(cleaned_paragraphs)

--Tu és livre. Partamos!

Vendo aproximar-se seu pai, D. Diogo de Mariz ergueu-se e descobrindo-se esperou-o n'uma attitude respeitosa.

Quando a corrente de ar expellio a fumaça do aposento, Cecilia pôde entrar e ver a scena que descrevêmos.

--Deixar-te! Uma figa! Depois de ter batido esta mattaria á tua procura! Tinha que ver!

--Pery vai fallar.

Number of paragraphs: 4892


4892

## Análise do dataset

In [8]:
# Conta as palavras no dataset
from collections import Counter
import re

def count_words(texts):
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(pattern, text.lower()))
    return word_counts

word_counts = count_words(cleaned_paragraphs)

len(word_counts)

12610

## Criando um vocabulário

In [9]:
most_frequent_words = [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}

In [10]:
print(f"Most Frequent Words: {most_frequent_words[:10]}")
print(f"Vocabulary Size: {len(vocab)}")

Most Frequent Words: ['.', ',', 'a', 'que', 'o', 'de', 'e', 'se', ';', 'um']
Vocabulary Size: 5000


In [11]:
def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in re.findall(pattern, sentence.lower())]

print(cleaned_paragraphs[20])
print(encode_sentence(cleaned_paragraphs[20], vocab))

 Publicando este livro em 1857, se disse ser aquella primeira edição uma prova typographica, que algum dia talvez o autor se dispuzesse a rever.
[0, 146, 4383, 23, 0, 2, 8, 50, 117, 276, 266, 2669, 13, 1071, 0, 2, 4, 193, 137, 287, 5, 2264, 8, 0, 3, 2672, 1]


## Classe do dataset

In [12]:
# Dataset class
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, paragraphs, vocab, context):
    self.paragraphs = paragraphs
    self.vocab = vocab
    self.context = context
    self.tokens, self.targets = self.setup()

  def __len__(self):
    return len(self.tokens)

  def __getitem__(self, idx):
    return torch.tensor(self.tokens[idx]), torch.tensor(self.targets[idx])
  
  def setup(self):
    tokens = []
    targets = []
    for paragraph in self.paragraphs:
      encoded = encode_sentence(paragraph, self.vocab)
      
      # If paragraph is smaller than the context, skip it.
      if len(encoded) < self.context + 1:
          continue

      for i in range(len(encoded) - self.context):
        tks = encoded[i:i+self.context]
        tgt = encoded[i+self.context]
        # Only add if there are no unknown tokens in both context and target.
        bad_token = 0
        if not (bad_token in tks or tgt == bad_token):
          tokens.append(tks)
          targets.append(tgt)
    return tokens, targets


In [13]:
# Train/Validation split
train_data, val_data = train_test_split(cleaned_paragraphs, test_size=0.2, random_state=18)

train_dataset = CustomDataset(train_data, vocab, context_size)
val_dataset = CustomDataset(val_data, vocab, context_size)

# Counting all Samples
print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print()
print(f"Training dataset samples: {len(train_dataset)}")
print(f"Validation dataset samples: {len(val_dataset)}")

Training samples: 3913
Validation samples: 979

Training dataset samples: 59646
Validation dataset samples: 16215


In [14]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

sample = next(iter(train_loader))
print(sample)

[tensor([[1428, 1884,    2,    6,  690],
        [ 196,    9,   15, 1053, 4023],
        [   5, 2019,   11,   19, 1036],
        [  18, 2938,    7, 3737,    3],
        [ 529,  127,  111, 2619,  289],
        [1406,   51,  244,    2,    7],
        [ 651,    4,   12,   32, 2519],
        [   3,  219,   51,  891,    8],
        [1189,    5,  293,  524,   11],
        [ 262, 2409,    3,  109,    7],
        [  12,   65,   24,   97,    6],
        [   3,   17,  481,    2, 1150],
        [  14,  386,   29, 2345,   63],
        [ 476, 4601, 4629,  125,    9],
        [ 695,   37, 1799,   18,    3],
        [2850,    6, 2125,    2,    4],
        [4530,   34,   39,  201, 4531],
        [  36,  786,   11,  510,    2],
        [ 901,    8,  424,    6,   10],
        [  40,   31,  299,    4,   70],
        [   4, 1918,    3,   35,    3],
        [ 142,  342, 3807,    2,  160],
        [   3,  358,   14,   57,    1],
        [2515,    5,  295,   18, 4248],
        [   1,    1,    1,    1,    1],

## Model

##### Implementação das camadas de self-attention

In [75]:
# Loop implementation
sample = next(iter(train_loader))
input = sample[0]
target = sample[1]

embeddings = nn.Embedding(vocab_size+1, embedding_dim)
embeds = embeddings(input)

batch_size, sequence_length, embedding_dimension = embeds.size()
print(f'Initial Embedding dimensions: {batch_size}, {sequence_length}, {embedding_dimension}')

# New embeddings
E = []
for q in range(sequence_length):
    scores = []
    C_q = embeds[:,q,:]
    for k in range(sequence_length):        
        C_k = embeds[:,k,:]
        
        # Scalar product between the embeddings
        score = torch.matmul(C_q, C_k.transpose(0,1))               
        scores.append(score)
    
    # Stack the matrixes resulting from the scalar products
    scores = torch.stack(scores, dim=1)
    probs = F.softmax(scores, dim=1)

    print(f'Scalar product dimensions: {scores.size()}')   
    print(f'Softmax from scalar products: {probs.size()}') 

    attn_embedding = torch.zeros_like(C_q)

    #for v, p in zip(range(sequence_length), probs):
    #    C_v = embeds[:,v,:]
    #    print(C_v.size())
    #    print(p.size())

    for v, p in zip(embeds, probs):
        v_q = v
        #attn_embedding += v * p2




    

    
    




Initial Embedding dimensions: 128, 5, 256
Scalar product dimensions: torch.Size([128, 5, 128])
Softmax from scalar products: torch.Size([128, 5, 128])
torch.Size([5, 128, 1])


RuntimeError: The size of tensor a (5) must match the size of tensor b (128) at non-singleton dimension 1

In [16]:
class SelfAttention_Loop(nn.Module):
  def __init__(self, dim):
    super().__init__()

  def forward(self, inputs):

    return 

In [17]:
class BengioModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, h):
        super(BengioModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size+1, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, h)
        self.relu = torch.nn.ReLU()
        self.linear2 = nn.Linear(h, vocab_size+1)
        # Softmax to scale outputs
        self.logSoftMax = torch.nn.LogSoftmax(dim=1)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        # Flatten embeddings
        embeds = embeds.view(embeds.size(0), -1)
        # Linear layer
        out = self.linear1(embeds)
        out = self.relu(out)
        # Second layer
        out = self.linear2(out)
        # Softmax output
        out = self.logSoftMax(out)
        return out

In [18]:
model = BengioModel(vocab_size, embedding_dim, context_size, hidden_dim)

In [19]:
sample = next(iter(train_loader))
input = sample[0]
target = sample[1]

print(input.shape)
print(target.shape)

torch.Size([128, 5])
torch.Size([128])


In [20]:
output = model(input)
print(output)
print(output.shape)

tensor([[-8.7484, -8.2451, -8.1889,  ..., -8.2732, -8.4269, -8.1852],
        [-8.3926, -8.8102, -8.2098,  ..., -8.5874, -8.8011, -8.3385],
        [-8.4746, -8.5855, -8.3107,  ..., -8.9544, -8.4054, -8.2674],
        ...,
        [-8.4366, -8.5997, -8.5453,  ..., -8.5630, -8.8300, -8.3888],
        [-8.6142, -8.3435, -8.4402,  ..., -8.6073, -8.4968, -8.6075],
        [-8.6048, -8.5584, -8.6162,  ..., -8.5801, -8.6487, -8.6064]],
       grad_fn=<LogSoftmaxBackward0>)
torch.Size([128, 5001])


In [21]:
output.argmax(dim=1)

tensor([ 865, 4225, 3402,  671, 3830, 1979, 4673, 2354,  135,  964, 2340,  540,
        1541, 2152, 4377, 2698,  646, 4376,  725, 1979,  725, 2565, 4431,  704,
        1598, 2406,  704, 3121, 3474, 3610, 3993, 1443,  753,   57, 2816, 4606,
        1632, 1888, 2354, 3296,  510, 1830, 1511,  834, 3309, 3289, 2752, 4326,
        2051, 1254, 1888, 2807, 4968, 3674,  133, 3462,  413, 2842,  113, 3989,
           9,  219, 4377, 2340,  473,  596, 3474,  842, 2354,  319,  475, 2340,
        2316,   26, 1226, 3296, 3296, 3121, 1961,  877,  257, 4338,  768, 1201,
        3340,  617,  239,  704, 2533, 1783, 3196,  113, 4158, 4944,  826, 4794,
        4820, 3253, 1729, 1979, 1888, 2050, 4604,  113, 3474, 3317,  358, 4326,
        4779, 2842, 2340,  228, 3679,  358, 1442,  540, 3032,  257, 1445, 3295,
         405, 3474,  467, 4377, 3295, 2050, 2340, 3293])

In [22]:
target

tensor([  15,   34,   28,   47,   65,  859,    7,   11,    2, 2079, 1666,  683,
          48,   32,  127,    1, 1626,   36,    2,   25,    1,   34,    9,  147,
          98,    9,    1,   24,  813,  849, 3439, 2882,    1, 1436,   77,  201,
           1, 1054,  646,    9, 3238,    4, 4314,   27,    6,   40,   15,    2,
           5,  685,    1, 1859,    4,   13,    7,  597, 4424,    1,  989, 1935,
           1,   48,    2,  945,   49,   29, 1454,  281,  248, 1865,  747,    4,
          14,   17,  249,   17,   10,   15, 2006,   18,    2,    4,   12, 1006,
           7,    5, 1701,  323,    1,    6,    1,  868,    1,  250,  134, 2329,
          97,  272, 3963,    1, 2684, 1044,  185,   81,    1,  314,  486,    2,
          96,    1,    7,   32,  250,    9, 1251, 4336,  318,    3,   36,   26,
         172,  108,  123,  147,    3,    7,  189, 3890])

## Training

In [23]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [24]:
# Cross Entropy
criterion = nn.CrossEntropyLoss()

# Optimizer
#optimizer = torch.optim.Adam(model.parameters(), lr)
optimizer = torch.optim.SGD(model.parameters(), lr)

model.to(device)

BengioModel(
  (embeddings): Embedding(5001, 256)
  (linear1): Linear(in_features=1280, out_features=128, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=128, out_features=5001, bias=True)
  (logSoftMax): LogSoftmax(dim=1)
)

In [25]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Exemplo de uso:
total_params = count_parameters(model)
print(f'O modelo tem um total de {total_params:,} parâmetros.')

O modelo tem um total de 2,089,353 parâmetros.


In [26]:
# Initial Perplexity and Loss
# Before training
model.eval()

loss = 0
perp = 0

with torch.no_grad():
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        loss += criterion(outputs, targets).item()

loss /= len(train_loader)
perp = torch.exp(torch.tensor(loss))

print(f'Initial Loss: {loss:.4f}')
print(f'Initial Perplexity: {perp:.4f}')

Initial Loss: 8.5289
Initial Perplexity: 5059.0957


In [27]:
# Training Loop
model.train()
for epoch in range(epochs):

  epoch_start = time.time()
  # Metrics
  epoch_loss = 0
  epoch_correct = 0
  epoch_samples = 0

  for inputs, targets in train_loader:
        inputs = inputs.to(device)  # Move input data to the device
        targets = targets.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Loss
        epoch_loss += loss.item()

        # Predicted
        _, predicted = torch.max(outputs, 1)
        epoch_correct += (predicted == targets).sum().item()
        epoch_samples += targets.size(0)

  # Calculate average loss and accuracy for epoch
  avg_loss = epoch_loss / len(train_loader)
  acc = epoch_correct / epoch_samples

  # Perplexity
  perp = torch.exp(torch.tensor(avg_loss))

  epoch_end = time.time()
  epoch_time = epoch_end - epoch_start
  # Print epoch statistics
  print(f'Epoch [{epoch+1}/{epochs}], Time:{epoch_time:.2f}, Loss: {avg_loss:.4f}, Accuracy: {acc:.2f}%, Perplexity: {perp:.4f}')


Epoch [1/10], Time:2.87, Loss: 6.4396, Accuracy: 0.11%, Perplexity: 626.1467
Epoch [2/10], Time:2.90, Loss: 5.5832, Accuracy: 0.15%, Perplexity: 265.9138
Epoch [3/10], Time:2.99, Loss: 5.2023, Accuracy: 0.18%, Perplexity: 181.6951
Epoch [4/10], Time:2.86, Loss: 4.9114, Accuracy: 0.20%, Perplexity: 135.8340
Epoch [5/10], Time:2.72, Loss: 4.6699, Accuracy: 0.22%, Perplexity: 106.6835
Epoch [6/10], Time:2.83, Loss: 4.4505, Accuracy: 0.24%, Perplexity: 85.6692
Epoch [7/10], Time:2.43, Loss: 4.2501, Accuracy: 0.26%, Perplexity: 70.1145
Epoch [8/10], Time:2.57, Loss: 4.0578, Accuracy: 0.28%, Perplexity: 57.8459
Epoch [9/10], Time:2.77, Loss: 3.8782, Accuracy: 0.30%, Perplexity: 48.3391
Epoch [10/10], Time:2.93, Loss: 3.7028, Accuracy: 0.32%, Perplexity: 40.5608


## Avaliação

In [28]:
model.eval()

loss_sum = 0
total_sum = 0
correct_sum = 0
eval_round = 0

loss = 0
perp = 0

with torch.no_grad():
    for inputs, targets in val_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, targets)      
        loss_sum += loss

        # Get the predicted labels
        _, predicted = torch.max(outputs, 1)

        total_sum += targets.size(0)
        correct_sum += (predicted == targets).sum().item()
        eval_round += 1

# Calculate accuracy
acc = 100 * correct_sum / total_sum

# Calculate average perplexity
average_loss = loss_sum / len(val_loader)
average_perplexity = torch.exp(average_loss)

print(f'Test Accuracy: {acc:.2f}%')
print(f'Average Loss: {average_loss:.2f}')
print(f'Average Perplexity: {average_perplexity:.2f}')

Test Accuracy: 19.41%
Average Loss: 5.36
Average Perplexity: 211.84


## Exemplo de uso

In [29]:
# Código adaptado da implementação do Cesar Bastos
from colorama import Fore, Style

text = cleaned_paragraphs
model.to(device)
def generate_text(model, vocab, text, max_length, context_size):
    words = []
    # Ensure there are enough words for at least one sequence
    while len(words) < context_size:
        random_number = random.randint(1, 4891)
        words = encode_sentence(text[random_number], vocab)
        if not words:
            words = []
            continue  # Skip if the sentence cannot be encoded
        words = words[:context_size]
        #print(words)
        if any(token == 0 for token in words):
            words = []
            continue  # Skip if any token is zero (assuming 0 is a special token)
        context = words

    print(f"Frase: {cleaned_paragraphs[random_number]}")
    print(words)

    for _ in range(max_length):
        words_tensor = torch.tensor(context[-context_size:], dtype=torch.long).unsqueeze(0).to(device)
        logits = model(words_tensor)
        probs = F.softmax(logits, dim=1)
        next_token = torch.multinomial(probs, num_samples=1)
        context.append(next_token.item())
        print(context)
    frase = []
    for i in context: ##Agradecimentos a Ramon Abilio
        word = next((word for word, code in vocab.items() if code == i), "<UNKNOWN>")
        frase.append(word)

    print(f"{Fore.BLUE}{frase[:context_size]}{Style.RESET_ALL} {Fore.RED}{frase[-max_length:]}{Style.RESET_ALL} ")


max_length= 10
generate_text(model, vocab, text, max_length, context_size)

Frase:  PAG. 111.--=Malvalisco=.
[239, 1, 2265, 1, 4931]
[239, 1, 2265, 1, 4931, 1]
[239, 1, 2265, 1, 4931, 1, 50]
[239, 1, 2265, 1, 4931, 1, 50, 54]
[239, 1, 2265, 1, 4931, 1, 50, 54, 106]
[239, 1, 2265, 1, 4931, 1, 50, 54, 106, 1242]
[239, 1, 2265, 1, 4931, 1, 50, 54, 106, 1242, 10]
[239, 1, 2265, 1, 4931, 1, 50, 54, 106, 1242, 10, 2663]
[239, 1, 2265, 1, 4931, 1, 50, 54, 106, 1242, 10, 2663, 5]
[239, 1, 2265, 1, 4931, 1, 50, 54, 106, 1242, 10, 2663, 5, 135]
[239, 1, 2265, 1, 4931, 1, 50, 54, 106, 1242, 10, 2663, 5, 135, 198]
['pag', '.', '111', '.', 'malvalisco'] ['.', 'disse', 'ella', 'esse', 'laço', 'um', 'whatsoever', 'o', 'fez', 'outros'] 
